# Projekt Python
Tomasz Ancukiewicz 127219, Mikołaj Leśny 127218

18.11.2019

## Init libs

In [1]:
import pandas as pd
from scipy.io import arff
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
import numpy as np

## Load data

In [2]:
SEED = 42

data, meta = arff.loadarff("data/breast-cancer.arff")
df = pd.DataFrame(data)

y = df.pop('Class').replace([b"'no-recurrence-events'", b"'recurrence-events'"],
                            ['no-recurrence-events', 'recurrence-events']).astype('str')
X = df
X = X.apply(lambda x: LabelEncoder().fit_transform(x))


## Make test and train sets

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=SEED)

## Classification

In [4]:
model = DecisionTreeClassifier(
    criterion='entropy',
    max_depth=7,
    min_samples_leaf=7,
    random_state=SEED,
    class_weight={"no-recurrence-events": 1.0, "recurrence-events": 5.0}
)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
print(f'confusion matrix\n{tp, fp}\n{fn, tn}')
print(f'accuracy: {accuracy_score(y_test, y_pred)}')
recal_recorrence_events = classification_report(y_test, y_pred, output_dict=True)["recurrence-events"]["recall"]
print(f'recal[recurrence-events]: {recal_recorrence_events}')


confusion matrix
(21, 31)
(3, 31)
accuracy: 0.6046511627906976
recal[recurrence-events]: 0.875
